In [1]:
%matplotlib inline

In [ ]:
# THIS FILE CONTAINS PREPROCESSING CODE THAT WAS USED ONCE AND IS SEPARATE TO
# IMPLY THAT IT DOESN'T NEED TO AGAIN UNLESS IMPORTANT FILES ARE DELETED

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import nltk
from collections import defaultdict
import re
import torch.utils.data as data
import sys
import os

from gensim import utils
from huffman_tree import HuffmanTree
from hierarchical_softmax import HierarchicalSoftmax
import hierarchical_softmax
import time
import copy

import unicodedata
import string
import json
import pickle
from copy import deepcopy

In [4]:
home_directory = os.path.expanduser('~')
nn_library_path = home_directory + '/Documents/neural_nets_research/Neural Nets Library'
sys.path.append(nn_library_path)

In [5]:
import training

In [ ]:
# USED TO CREATE FREQUENCY DICTIONARY (ran once)

wikipedia_text_path = "wiki.en.text"

# Read line by line
last_line = None
frequencies = defaultdict(int)
all_letters = string.ascii_letters

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

counter = 1
with open(wikipedia_text_path) as f:
    for line in f:
        for word in line.split():
            frequencies[unicodeToAscii(word)] += 1
            
        if (counter % 10000 == 0):
            print(counter)

        counter += 1

filter_file_handle = open("wiki.en.frequencies.pkl", "wb")
pickle.dump(frequencies, filter_file_handle)

In [ ]:
# Test loading of wikipedia into a dictionary
def printOccurrence(dictionary, word):
    print("The word", word, "appears", dictionary[word], "times.")
    
printOccurrence(frequencies, "mehdi")
printOccurrence(frequencies, "ununpentium")
print(len(frequencies))

In [ ]:
# USED TO CREATE WIKIPEDIA SUBFILES (ran once)

In [ ]:
def make_filtered_subfiles(large_file_path, filter_dictionary, divisor_value):
    file_handle = open(large_file_path, "r")
    file_counter = 0
    subfiles_handle_prefix = "/media/mehdi2277/MyFiles/large_datasets/wikipedia/wiki_s_"
    
    line_counter = 0
    current_subfile_array = []
    with open(large_file_path) as f:
        for line in f:
            for word in line.split():
                if word in filter_dictionary:
                    current_subfile_array += [word]

            if (line_counter % divisor_value == 0):
                subfile_name = subfiles_handle_prefix + str(file_counter) + ".json"
                subfile_handle = open(subfile_name, "w")
                
                # Save serializable array of words corresponding to 1000 articles
                json.dump(current_subfile_array, subfile_handle)
                print("Writing to ", subfile_name)
                
                # Increment file count and close file
                file_counter += 1
                subfile_handle.close()
                
                # Empty array
                current_subfile_array = []

            line_counter += 1

In [ ]:
threshold = 120
wikipedia_text_path = "wiki.en.text"
frequencies = {k: v for k, v in frequencies.items() if v > threshold}

# Create subfiles
make_filtered_subfiles(wikipedia_text_path, frequencies, 1000)

In [ ]:
# Reverse word to id dictionary
id_to_word = []
id_to_word_dict = {}

for word, id_value in word_dictionary.items():
    id_to_word_dict[id_value] = word

for i in range(len(id_to_word_dict)):
    id_to_word.append(id_to_word_dict[i])